In [1]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy
%reload_ext autoreload
%autoreload 

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

In [4]:
from nes_py.wrappers import JoypadSpace

In [5]:
import gym_super_mario_bros
import gym_super_mario_bros.actions as JoypadActions

from lib.env_wrappers import EnvWrapperFactory

stateShape = (50, 50)
actionShape = len(JoypadActions.SIMPLE_MOVEMENT)

In [6]:
%reload_ext autoreload
%autoreload 
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, JoypadActions.SIMPLE_MOVEMENT)
env = EnvWrapperFactory.convert(env, shape=stateShape)
state = env.reset()
print(state.shape)

shape before any transformations: (240, 256, 3)
shape after grayscaler: (240, 256)
shape after resizer: (50, 50)
shape after all transformations: (5, 50, 50)
(5, 50, 50)


In [7]:
from agents.ForgetfulAgent import ForgetfulAgent
%reload_ext autoreload
%autoreload 

In [8]:
env.observation_space.shape

(5, 50, 50)

In [9]:
%reload_ext autoreload
%autoreload 
agent = ForgetfulAgent(state_shape=env.observation_space.shape, action_shape=actionShape, device=device)

In [10]:
# try training

maxSteps = 10000
done = True
learnCount = 0

for i in range(maxSteps):
    if done:
        state = env.reset()
    
    action = agent.getAction(state)
    next_state, reward, done, info = env.step(env.action_space.sample())
    
    # add to memory
    agent.cache(state, next_state, action, reward, done)
    
    q, loss = agent.learn()
    
    
    if q is not None:
        learnCount += 1
        if learnCount % 100:
            print(f"q:{q}, loss={loss}")

q:0.026198601350188255, loss=0.21926084160804749
q:0.6211857795715332, loss=0.6312114000320435
q:0.27534550428390503, loss=0.5436283349990845
q:0.2912709712982178, loss=0.39244344830513
q:0.28735125064849854, loss=0.4541400671005249
q:0.32353347539901733, loss=0.4170064628124237
q:0.4117383360862732, loss=0.27786481380462646
q:0.34249943494796753, loss=0.2962423861026764
q:0.25497961044311523, loss=0.3174841105937958
q:0.21846255660057068, loss=0.30174094438552856
q:0.2077147364616394, loss=0.32695281505584717
q:0.2542179226875305, loss=0.5210808515548706
q:0.344338595867157, loss=0.332788348197937
q:0.3804614543914795, loss=0.488013356924057
q:0.44692087173461914, loss=0.25771549344062805
q:0.4628807306289673, loss=0.393270879983902
q:0.395152747631073, loss=0.5370404720306396
q:0.40172868967056274, loss=0.40506109595298767
q:0.30543506145477295, loss=0.4653308093547821
q:0.30082571506500244, loss=0.3255654573440552
q:0.233079195022583, loss=0.2830992043018341
q:0.22229944169521332, l

KeyboardInterrupt: 

In [17]:
states, next_states, actions, rewards, dones = agent.sampleExperienceBatch()

In [12]:
states.shape

torch.Size([32, 5, 50, 50])

In [13]:
actions.shape

torch.Size([32])

In [14]:
rewards.shape

torch.Size([32])

In [15]:
dones.shape

torch.Size([32])

In [19]:
print(rewards)

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,
        -2,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       device='cuda:0', dtype=torch.int32)
